In [1]:
import sys
import os
%matplotlib inline


In [2]:
os.getcwd()

'c:\\Users\\gabi1\\git\\ACTC_MD_ML\\gaby_notebooks'

In [3]:
os.chdir('..\\')
os.getcwd()

'c:\\Users\\gabi1\\git\\ACTC_MD_ML'

In [4]:
from rdkit.Chem.Draw import rdMolDraw2D
from src.trainers import Trainers
from src.polymerSMILES import polymers
from src.dataProcessing import * #DataPreprocessing #computeTanimotoSimilarity #* #data_load, computeTanimotoSimilarity, get_dissimilarity_map, featurizer, normalizedata
from src.solver import *

# Data preprocessing part

### Load polymer smiles

In [5]:
# load smiles
polymers_dict = polymers.get_data()

### Implement datapreprocessing class

In [6]:
# data preprocessing toolbox
preprocess = DataPreprocessing()

### Loading the data

- data1 will be use for activity

In [7]:
data1 = preprocess.data_load(filename ='1-0-hgk-MD-ActCoeff-processed_Activity.csv')
data1.head()

,#,Name of the polymer,CounterIon,Co-Ion,salt,Water-per-ion,concentration of salt (M),gr_minima (Ion_H2O),gr_peak_position (Ion_H2O),gr_peak_height (Ion_H2O),Nr_(Ion_H2O),gr_minima (CG_H2O),gr_peak_position (CG_H2O),gr_peak_hight (CG_H2O),Nr_(CG_H2O),gr_minima (CG_Ion),gr_peak_position (CG_Ion),gr_peak_height (CG_Ion),Nr_(CG_Ion),Experimental activity co.
0,1,PSbNMP_RCE,I-,K+,KI,6,0.02,4.45,3.65,6.50,4.70,5.50,3.55,2.30,6.10,6.15,4.15,5.80,1.90,0.300000
1,2,PSbNMP_RCE,I-,K+,KI,6,0.20,4.45,3.65,6.30,4.60,5.50,3.55,1.90,5.00,6.15,4.15,5.80,2.00,0.319000
2,3,PSbNMP_RCE,I-,K+,KI,6,0.50,4.45,3.65,6.20,4.60,5.50,3.55,1.80,4.50,6.15,4.15,5.60,2.20,0.476000
3,4,PSbNMP_RCE,I-,K+,KI,6,1.00,4.55,3.65,6.00,4.70,5.00,3.55,1.50,3.00,6.15,4.15,5.10,2.50,0.663000
4,5,PVAS_RCE,Na+,Cl-,NaCl,113,0.02,3.15,2.35,8.55,5.38,3.37,2.76,1.83,2.98,3.06,2.35,6.71,0.13,0.280899


- data2 will be use for solvation

In [8]:
data2 = preprocess.data_load(filename = "1.0-hkg-MD-ActCoeff-processed.csv")
data2.head()

,#,Name of the polymer,CounterIon,Co-Ion,salt,Water-per-ion,concentration_of_salt_M,gr_minima_Ion_H2O,gr_peak_position_Ion_H2O,gr_peak_height_Ion_H2O,Nr_Ion_H2O,gr_minima_CG_H2O,gr_peak_position_CG_H2O,gr_peak_height_CG_H2O,Nr_CG_H2O_,gr_minima_CG_Ion,gr_peak_position_CG_Ion,gr_peak_height_CG_Ion,Nr_CG_Ion,Exp_act_coeff
0,1,PSbNMP_RCE,I-,K+,KI,6,0.00,4.45,3.65,6.5,4.7,5.4,3.55,2.16,5.3,6.25,4.15,5.6,1.9,-
1,2,PSbNMP_RCE,I-,K+,KI,6,0.02,4.45,3.65,6.5,4.7,5.5,3.55,2.30,6.1,6.15,4.15,5.8,1.9,0.300
2,3,PSbNMP_RCE,I-,K+,KI,6,0.20,4.45,3.65,6.3,4.6,5.5,3.55,1.90,5.0,6.15,4.15,5.8,2.0,0.319
3,4,PSbNMP_RCE,I-,K+,KI,6,0.50,4.45,3.65,6.2,4.6,5.5,3.55,1.80,4.5,6.15,4.15,5.6,2.2,0.476
4,5,PSbNMP_RCE,I-,K+,KI,6,1.00,4.55,3.65,6.0,4.7,5.0,3.55,1.50,3.0,6.15,4.15,5.1,2.5,0.663


### Featurizer the data 

- this is going to define X and y depending on the descr.

#### For activity

In [9]:
## assign X1, y1 when descr = activity 
(X1, y1, MFprints) = preprocess.featurizer(data1, polymers_dict,  'activity') # descr = 'solvation' or 'activity'

In [10]:
#since the descr == activity X should be reverse_fingerprint + Ohe_df and data_mm(mixcared)
X1

,mfp_0,mfp_1,mfp_2,mfp_3,mfp_4,mfp_5,mfp_6,mfp_7,mfp_8,mfp_9,...,E,F,G,H,I,J,K,L,M,N
0,0,1,0,1,0,1,1,0,0,0,...,-0.940850,-0.022222,0.614035,-0.028571,-0.324042,-0.465909,0.937322,0.525424,-0.865209,0.514170
1,0,1,0,1,0,1,1,0,0,0,...,-0.953173,-0.066667,0.614035,-0.028571,-0.602787,-0.590909,0.937322,0.525424,-0.865209,0.595142
2,0,1,0,1,0,1,1,0,0,0,...,-0.959335,-0.066667,0.614035,-0.028571,-0.672474,-0.647727,0.937322,0.525424,-0.872495,0.757085
3,0,1,0,1,0,1,1,0,0,0,...,-0.971657,-0.022222,0.263158,-0.028571,-0.881533,-0.818182,0.937322,0.525424,-0.890710,1.000000
4,0,1,0,1,0,1,1,0,0,0,...,-0.814541,0.280000,-0.880702,-0.931429,-0.651568,-0.820455,-0.823362,-0.694915,-0.832058,-0.919028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1,1,1,0,1,1,0,0,1,0,...,-0.736907,-0.057778,-0.838596,-0.942857,-0.749129,-0.929545,-0.851852,-0.694915,-0.795264,-0.910931
76,1,1,1,0,1,1,0,0,1,0,...,-0.815773,-0.315556,-0.719298,-0.942857,-0.909408,-0.886364,-0.829060,-0.694915,-0.543169,-0.773279
77,1,1,1,0,1,1,0,0,1,0,...,-0.743685,-0.066667,-0.754386,-0.942857,-0.756098,-0.894318,-0.829060,-0.694915,-0.774135,-0.878543
78,1,1,1,0,1,1,0,0,1,0,...,-0.749846,-0.102222,-0.754386,-0.942857,-0.770035,-0.901136,-0.823362,-0.694915,-0.765027,-0.870445


In [11]:
#y should be the activity coef column only
y1

0     0.300000
1     0.319000
2     0.476000
3     0.663000
4     0.280899
        ...   
75    0.600412
76    0.685773
77    0.667216
78    0.704330
79    0.819381
Name: Experimental activity co., Length: 80, dtype: float64

#### For solvation

In [12]:
## assign X2, y2 when descr = solvation 
(X2, y2, MFprints) = preprocess.featurizer(data2, polymers_dict,  'solvation') # descr = 'solvation' or 'activity'

In [13]:
# X2 contains reverse_fingerprint + Ohe_df + water per ion and concentration of salt
X2

,mfp_0,mfp_1,mfp_2,mfp_3,mfp_4,mfp_5,mfp_6,mfp_7,mfp_8,mfp_9,...,ohe_3,ohe_4,ohe_5,ohe_6,ohe_7,ohe_8,ohe_9,ohe_10,Water-per-ion,concentration_of_salt_M
0,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.00
1,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.02
2,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.20
3,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.50
4,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,1,1,0,1,1,1,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,24,0.01
88,1,1,0,1,1,1,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,24,0.03
89,1,1,0,1,1,1,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,24,0.10
90,1,1,0,1,1,1,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,24,0.30


In [14]:
# y2 contains the rest of the parameter in the data2
y2

,gr_minima_Ion_H2O,gr_peak_position_Ion_H2O,gr_peak_height_Ion_H2O,Nr_Ion_H2O,gr_minima_CG_H2O,gr_peak_position_CG_H2O,gr_peak_height_CG_H2O,Nr_CG_H2O_,gr_minima_CG_Ion,gr_peak_position_CG_Ion,gr_peak_height_CG_Ion,Nr_CG_Ion
0,4.45,3.65,6.50,4.70,5.40,3.55,2.16,5.30,6.25,4.15,5.60,1.90
1,4.45,3.65,6.50,4.70,5.50,3.55,2.30,6.10,6.15,4.15,5.80,1.90
2,4.45,3.65,6.30,4.60,5.50,3.55,1.90,5.00,6.15,4.15,5.80,2.00
3,4.45,3.65,6.20,4.60,5.50,3.55,1.80,4.50,6.15,4.15,5.60,2.20
4,4.55,3.65,6.00,4.70,5.00,3.55,1.50,3.00,6.15,4.15,5.10,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...
87,3.18,2.35,9.81,4.62,3.43,2.75,1.69,2.02,3.01,2.35,7.72,0.14
88,3.16,2.35,8.53,4.04,3.60,2.75,1.46,2.40,3.05,2.35,14.64,0.31
89,3.17,2.35,9.70,4.60,3.55,2.75,1.68,2.33,3.05,2.35,8.30,0.18
90,3.15,2.35,9.60,4.52,3.55,2.75,1.66,2.27,3.06,2.35,8.55,0.19


### Split and normailize data

#### Activity

In [15]:
(data_activity_scaler, X_train1, X_test1, y_train1, y_test1) = preprocess.normalizedata(X1, y1, splitRatio=0.2, state = 48, transform = True, property = None)

#### Solvation

In [16]:
(data_solvation_scaler, X_train2, X_test2, y_train2, y_test2) = preprocess.normalizedata(X2, y2, splitRatio=0.2, state = 48, transform = True, property = 'Y')

### PCA transformation for A1 and A2

Reminder the the input vestors:
- A0 represents all data with no transformation
- A1 PCA applied only to the polymer fingerprint
- A2 PCE applied to all input features

#### Activity

##### A1 (MDFP represent the polymer fingerprint input)

In [17]:
pca_X1_train_MDFP, pca_X1_test_MDFP = preprocess.convertInputsPCA(X_train1, X_test1, MFprints, n_comp = 15, type = 'MDFP')

##### A2 (ALL represent all input)

In [18]:
pca_X1_train_ALL, pca_X1_test_ALL = preprocess.convertInputsPCA(X_train1, X_test1, MFprints, n_comp = 15, type = 'All')

#### Solvation

##### A1

In [19]:
pca_X2_train_MDFP, pca_X2_test_MDFP = preprocess.convertInputsPCA(X_train2, X_test2, MFprints, n_comp=10, type = 'MDFP')

##### A2

In [20]:
pca_X2_train_ALL, pca_X2_test_ALL = preprocess.convertInputsPCA(X_train2, X_test2, MFprints, n_comp=10, type = 'All')

# ML models

## Support Vector Regression (SVR)

## Artificial Neural Network (ANN)

## Random Forest Regression (RFR)